In [6]:
import pandas as pd
import re
import spacy
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
import numpy as np

In [7]:
tulu_df=pd.read_csv("/content/Tulu_SA_train (1).csv")
tulu_dev=pd.read_csv("/content/Tulu_SA_val (1).csv")
tulu_test=pd.read_csv("/content/Tulu_SA_test_without_label (1).csv")

In [8]:
tulu_train=pd.concat([tulu_df, tulu_dev], ignore_index=True)
tulu_train.head()

,Text,Label
0,Aunty log bohot kadak hai,Not Tulu
1,Shruthi was awesome... Nice collaboration... H...,Not Tulu
2,Gol gappadh ammana sajjigene best,Positive
3,Chaddida brand thojodijji marre😃😃😃,Neutral
4,Memories just got refreshed...,Not Tulu


#Cleaning the training dataset

In [9]:
!pip install contractions
#CLEANING THE DEVELOPED TULU
tulu_train = tulu_train.dropna(subset=['Label'])
import re

# Apply the cleaning logic to the "Text" column
tulu_train["Text"] = tulu_train["Text"].apply(lambda x: re.sub(r'\s+', ' ', str(x).strip()))

# Convert the "Text" column to lowercase
tulu_train["Text"] = tulu_train["Text"].str.lower()
import re
import string

# Clean, remove punctuation, and convert to lowercase
tulu_train["Text"] = tulu_train["Text"].apply(
    lambda x: re.sub(r'\s+', ' ', ''.join(char for char in str(x) if char not in string.punctuation).strip()).lower()
)


import re

# Function to clean text
def clean_text(text):
    # Remove emojis using regex
    text = re.sub(r'[^\w\s,]', '', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply the cleaning function to the "Text" column
tulu_train["Text"] = tulu_train["Text"].apply(clean_text)


from nltk.corpus import stopwords

# Download stopwords if not already downloaded
import nltk
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

# Remove stopwords
tulu_train["Text"] = tulu_train["Text"].apply(
    lambda x: ' '.join(word for word in x.split() if word not in stop_words)
)


from nltk.stem import WordNetLemmatizer

# Download WordNet data if not already downloaded
nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

# Lemmatize each word in the text
tulu_train["Text"] = tulu_train["Text"].apply(
    lambda x: ' '.join(lemmatizer.lemmatize(word) for word in x.split())
)

import contractions

# Expand contractions
tulu_train["Text"] = tulu_train["Text"].apply(lambda x: contractions.fix(x))

custom_stopwords = {"super", "vlog", "gameplay"}

tulu_train["Text"] = tulu_train["Text"].apply(
    lambda x: ' '.join(word for word in x.split() if word not in custom_stopwords)
)

# Replace negations
tulu_train["Text"] = tulu_train["Text"].apply(
    lambda x: re.sub(r'\bnot (\w+)', r'not_\1', x)
)

# Remove repeated characters
tulu_train["Text"] = tulu_train["Text"].apply(
    lambda x: re.sub(r'(.)\1+', r'\1\1', x)
)

# Remove non-ASCII characters
tulu_train["Text"] = tulu_train["Text"].apply(lambda x: x.encode('ascii', 'ignore').decode('utf-8'))

tulu_train = tulu_train.dropna(subset=['Text'])
tulu_train = tulu_train.dropna(subset=['Text'])

import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

# Define stopwords
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-alphabetic characters
    text = " ".join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    return text

# Apply cleaning function to the 'Text' column
tulu_train['cleaned_text'] = tulu_train['Text'].apply(clean_text)

tulu_train[tulu_train['cleaned_text'].str.strip() == '']  # Find rows with empty cleaned_text

# Drop rows with empty 'cleaned_text'
tulu_train = tulu_train[tulu_train['cleaned_text'].str.strip() != '']  # Remove rows where cleaned_text is empty

import re

# Function to remove repeating characters from any word
def remove_repeated_chars(text):
    return re.sub(r'(.)\1+', r'\1', text)

# Apply function to remove repeated characters in the entire column
tulu_train['cleaned_text'] = tulu_train['cleaned_text'].apply(remove_repeated_chars)

# Verify the column has been created
print(tulu_train.head())  # Check the first few rows to confirm the new column



<ipython-input-9-bfff1a99b97e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tulu_train["Text"] = tulu_train["Text"].apply(lambda x: re.sub(r'\s+', ' ', str(x).strip()))
<ipython-input-9-bfff1a99b97e>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tulu_train["Text"] = tulu_train["Text"].str.lower()
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package o

                                                Text     Label  \
0                          aunty log bohot kadak hai  Not Tulu   
1  shruthi awesome nice collaboration hope see yo...  Not Tulu   
2                  gol gappadh ammana sajjigene best  Positive   
3                    chaddida brand thojodijji marre   Neutral   
4                               memory got refreshed  Not Tulu   

                                        cleaned_text  
0                          aunty log bohot kadak hai  
1  shruthi awesome nice colaboration hope se upco...  
2                     gol gapadh amana sajigene best  
3                       chadida brand thojodiji mare  
4                               memory got refreshed  


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-9-bfff1a99b97e>:118: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tulu_train['cleaned_text'] = tulu_train['cleaned_text'].apply(remove_repeated_chars)


#Cleaning Testing dataset

In [10]:
!pip install contractions

import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import contractions
import nltk

# Download necessary data
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


# Initial preprocessing: clean and normalize text
tulu_test["Text"] = tulu_test["Text"].apply(lambda x: re.sub(r'\s+', ' ', str(x).strip()))
tulu_test["Text"] = tulu_test["Text"].str.lower()
tulu_test["Text"] = tulu_test["Text"].apply(
    lambda x: re.sub(r'\s+', ' ', ''.join(char for char in str(x) if char not in string.punctuation).strip()).lower()
)

# Function to clean text
def clean_text(text):
    text = re.sub(r'[^\w\s,]', '', text)  # Remove emojis and special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace
    return text

tulu_test["Text"] = tulu_test["Text"].apply(clean_text)

# Remove stopwords
stop_words = set(stopwords.words('english'))
tulu_test["Text"] = tulu_test["Text"].apply(
    lambda x: ' '.join(word for word in x.split() if word not in stop_words)
)

# Lemmatization
lemmatizer = WordNetLemmatizer()
tulu_test["Text"] = tulu_test["Text"].apply(
    lambda x: ' '.join(lemmatizer.lemmatize(word) for word in x.split())
)

# Expand contractions
tulu_test["Text"] = tulu_test["Text"].apply(lambda x: contractions.fix(x))

# Remove custom stopwords
custom_stopwords = {"super", "vlog", "gameplay"}
tulu_test["Text"] = tulu_test["Text"].apply(
    lambda x: ' '.join(word for word in x.split() if word not in custom_stopwords)
)

# Replace negations
tulu_test["Text"] = tulu_test["Text"].apply(
    lambda x: re.sub(r'\bnot (\w+)', r'not_\1', x)
)

# Remove repeated characters
tulu_test["Text"] = tulu_test["Text"].apply(
    lambda x: re.sub(r'(.)\1+', r'\1\1', x)
)

# Define a new cleaned column without dropping rows
tulu_test['cleaned_text'] = tulu_test["Text"].apply(clean_text)

# Remove repeating characters in the cleaned text
def remove_repeated_chars(text):
    return re.sub(r'(.)\1+', r'\1', text)

tulu_test['cleaned_text'] = tulu_test['cleaned_text'].apply(remove_repeated_chars)

# Ensure no rows are dropped
print(len(tulu_test))  # Length of the dataframe after processing


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


1479


#SVM MODEL

In [11]:
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.utils.class_weight import compute_class_weight

vectorizer = TfidfVectorizer(max_features=5000)
# Separate features and labels
X_train = vectorizer.fit_transform(tulu_train['cleaned_text'])  # Assuming vectorized cleaned text
y_train = tulu_train['Label']

# Apply SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)



# Compute class weights
class_weights = compute_class_weight('balanced', classes=y_train.unique(), y=y_train)
class_weight_dict = dict(zip(y_train.unique(), class_weights))

# Train the model with class weights
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', class_weight=class_weight_dict, random_state=42)
svm_model.fit(X_train, y_train)


# Vectorize the test data
X_test = vectorizer.transform(tulu_test['cleaned_text'])
#y_test = tulu_train['Label']

# Predict on test data
y_pred = svm_model.predict(X_test)
tulu_test['Label']=y_pred

# Check the updated DataFrame
tulu_test.drop(columns=['Text'], inplace=True)
tulu_test.drop(columns=['cleaned_text'], inplace=True)

tulu_test.head()

print(tulu_test.head())



         Id     Label
0  SA_TU_01  Not Tulu
1  SA_TU_02  Not Tulu
2  SA_TU_03  Not Tulu
3  SA_TU_04   Neutral
4  SA_TU_05   Neutral


In [12]:
print(tulu_test)

              Id     Label
0       SA_TU_01  Not Tulu
1       SA_TU_02  Not Tulu
2       SA_TU_03  Not Tulu
3       SA_TU_04   Neutral
4       SA_TU_05   Neutral
...          ...       ...
1474  SA_TU_1475  Negative
1475  SA_TU_1476     Mixed
1476  SA_TU_1477  Not Tulu
1477  SA_TU_1478   Neutral
1478  SA_TU_1479   Neutral

[1479 rows x 2 columns]
